<a href="https://colab.research.google.com/github/eistratova/PyTorch_programming_lessons/blob/main/Programming_PyTorch_05_Katya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Programming PyTorch for Deep Learning
Creating and Deploying Deep Learning Applications"

Ian Pointer

Ссылка на материалы в GitHub: https://github.com/falloutdurham/beginners-pytorch-deep-learning

Руководство по работе в Google Colab смотрите по [ссылке]( https://colab.research.google.com/notebooks/intro.ipynb#scrollTo=GJBs_flRovLc)

---



# Глава 5. Классификация текстов.
## Сентимент анализ твитов
Дата-сеты для упражнения по классификации (сентимент анализ) твитов находятся по [ссылке](http://help.sentiment140.com/for-students).

In [2]:
# Подключение к Google-диску, где хранятся анные и будут сохраняться модели.
# При запуске модуля будет показана ссылка - кликайте на неё и входите через свой аккаунт.
# Копируйте ключ и вставьте его в окошко...
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [3]:
# Проверяем наличие доступа к директории, в которой будет сохраняться модель
%cd '/gdrive/MyDrive/Colab_Notebooks/text_classification/trainingandtestdata'
%ls

/gdrive/MyDrive/Colab_Notebooks/text_classification/trainingandtestdata
testdata.manual.2009.06.14.csv             train-processed.csv
training.1600000.processed.noemoticon.csv


In [4]:
dataset_dir = "/gdrive/MyDrive/Colab_Notebooks/text_classification/trainingandtestdata/"
ready_data_dir = "/gdrive/MyDrive/Colab_Notebooks/text_classification/preprocessed_data/"
# ready_data_dir = "/content/"

In [5]:
# Pandas для предобработки данных
import pandas as pd
tweetsDF = pd.read_csv(dataset_dir+"training.1600000.processed.noemoticon.csv",engine="python", header=None)

In [6]:
print(tweetsDF[0].value_counts())
tweetsDF.head(5)

4    800000
0    800000
Name: 0, dtype: int64


,0,1,2,3,4,5
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [7]:
#  Чтобы кодировать классы как числа, начинающиеся с 0, мы сначала создаем столбец типа category из столбца маркировки
tweetsDF["sentiment_cat"] = tweetsDF[0].astype('category')
# Затем мы кодируем эти классы в виде числовой информации в другом столбце:
tweetsDF["sentiment"] = tweetsDF["sentiment_cat"].cat.codes


In [8]:
# Смотрим, что получилось
tweetsDF.head(5)


,0,1,2,3,4,5,sentiment_cat,sentiment
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",0,0
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,0,0
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,0,0
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,0,0
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",0,0


In [9]:
# Смотрим, что получилось
print(tweetsDF.tail(5))

         0           1  ... sentiment_cat sentiment
1599995  4  2193601966  ...             4         1
1599996  4  2193601969  ...             4         1
1599997  4  2193601991  ...             4         1
1599998  4  2193602064  ...             4         1
1599999  4  2193602129  ...             4         1

[5 rows x 8 columns]


In [10]:
# Затем мы сохраняем измененный CSV в файл:
tweetsDF.to_csv(ready_data_dir+"train-processed.csv", header=None, index=None)
# Для быстрого тестирования выделяем небольшой дата-сет
tweetsDF.sample(1000000).to_csv(ready_data_dir+"train-processed-sample.csv", header=None,index=None)

Библиотека **torchtext** генерирует набор данных простым способом: вы говорите, что вам нужно, и она обрабатывает исходный CSV (или JSON). Для этого сначала определяем поля. Класс `Field` имеет значительное количество параметров, которые могут быть ему присвоены: https://torchtext.readthedocs.io/en/latest/data.html#field

In [11]:
import spacy
import torch

import torchtext
import torch.nn as nn
import torch.optim as optim

# from torchtext import data # так в книге, но тогда выдаётся ошибка для data.LabelField()
# Работает нижеследующий вариант импорта модуля data
from torchtext.legacy import data
# в докумнтации Pyorch про legacy ничего нет
# как я понял, в torchtext.legacy, находится старая версия модуля data

In [12]:
%%time
# Нас интересуют только маркировки и текст твитов. Мы определяем их, используя тип данных Field
LABEL = data.LabelField()
TWEET = data.Field(tokenize='spacy', tokenizer_language='en_core_web_sm', lower=True) # в книге нет tokenizer_language='en_core_web_sm'

CPU times: user 539 ms, sys: 33 ms, total: 572 ms
Wall time: 789 ms


# TabularDataset
`class torchtext.data.TabularDataset(path, format, fields, skip_header=False, csv_reader_params={}, **kwargs)`

Defines a Dataset of columns stored in CSV, TSV, or JSON format.

`__init__(path, format, fields, skip_header=False, csv_reader_params={}, **kwargs)`

Create a TabularDataset given a path, file format, and field list.

Parameters:	
* **path** (*str*) – Path to the data file.
* **format** (*str*) – The format of the data file. One of “CSV”, “TSV”, or “JSON” (case-insensitive).
* **fields** (*list(tuple(str, Field)*) – tuple(str, Field)]: If using a list, the format must be CSV or TSV, and the values of the list should be tuples of (name, field). The fields should be in the same order as the columns in the CSV or TSV file, while tuples of (name, None) represent columns that will be ignored.

If using a dict, the keys should be a subset of the JSON keys or CSV/TSV columns, and the values should be tuples of (name, field). Keys not present in the input dictionary are ignored. This allows the user to rename columns from their JSON/CSV/TSV key names and also enables selecting a subset of columns to load.

* **skip_header** (*bool*) – Whether to skip the first line of the input file.
* **csv_reader_params** (*dict*) – Parameters to pass to the csv reader. Only relevant when format is csv or tsv. See https://docs.python.org/3/library/csv.html#csv.reader for more details.

In [13]:
%%time
# После определения полей LABEL и TWEET нам нужно создать список, который свяжет их со списком строк в CSV
fields = [('score',None), ('id',None),('date',None),('query',None),
          ('name',None),('tweet', TWEET),('category',None),('label',LABEL)]

# Вооружившись объявленными полями, мы используем TabularDataset, чтобы применить это определение к CSV:
twitterDataset = data.TabularDataset(
        path = ready_data_dir+"train-processed-sample.csv",
        fields = fields,
        format = "csv",
        skip_header=False)

CPU times: user 2min 32s, sys: 1.74 s, total: 2min 33s
Wall time: 2min 33s


In [14]:
# делим наборы данных на обучение, тестирование и верификацию с помощью метода split():
(train, test, valid) = twitterDataset.split(split_ratio=[0.6,0.2,0.2])
(len(train),len(test),len(valid))

(600000, 200000, 200000)

In [15]:
# пример, вытащенный из набора данных:
vars(train.examples[4000])

{'label': '1',
 'tweet': ['@abcoates',
  'was',
  'talking',
  'about',
  'the',
  'swiftcommunity.net',
  'homepage',
  ' ',
  '...',
  'nice',
  'idea',
  'thought',
  '...']}



---



In [16]:
# создание словаря
vocab_size = 30000
TWEET.build_vocab(train, max_size = vocab_size)
LABEL.build_vocab(train)
TWEET.vocab.freqs.most_common(10)

[('i', 373546),
 ('!', 338428),
 ('.', 303988),
 (' ', 219864),
 ('to', 212227),
 ('the', 195814),
 (',', 181300),
 ('a', 142744),
 ('my', 118483),
 ('it', 113740)]

In [17]:
%%time

device = "cuda"
# создаём загрузчик данных
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train, valid, test),
    batch_size = 32,
    device = device,
    sort_key = lambda x: len(x.tweet),
    sort_within_batch = False)

CPU times: user 541 µs, sys: 0 ns, total: 541 µs
Wall time: 602 µs


## Our First LSTM

In [18]:
# Создаём сеть из 3-ёх слоёв
device = "cuda"
class OurFirstLSTM(nn.Module):
    def __init__(self, hidden_size, embedding_dim, vocab_size):
        super(OurFirstLSTM, self).__init__()
    
        self.embedding = nn.Embedding(vocab_size, embedding_dim) # 1-st layer wih dimention=300 (see below)
        self.encoder = nn.LSTM(input_size=embedding_dim,  # one layer of the LSTM 
                hidden_size=hidden_size, num_layers=1)    # with dimention=100 (see below)
        self.predictor = nn.Linear(hidden_size, 2)        # 3d  layer with 2 output

    def forward(self, seq):
        output, (hidden,_) = self.encoder(self.embedding(seq))
        preds = self.predictor(hidden.squeeze(0))
        return preds

model = OurFirstLSTM(100,300, 30002)
model.to(device)

OurFirstLSTM(
  (embedding): Embedding(30002, 300)
  (encoder): LSTM(300, 100)
  (predictor): Linear(in_features=100, out_features=2, bias=True)
)

## Training

In [19]:
optimizer = optim.Adam(model.parameters(), lr=2e-2)
criterion = nn.CrossEntropyLoss()

def train(epochs, model, optimizer, criterion, train_iterator, valid_iterator):
    for epoch in range(1, epochs+1):
     
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch_idx, batch in enumerate(train_iterator):
            optimizer.zero_grad()
            predict = model(batch.tweet)
            loss = criterion(predict,batch.label)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * batch.tweet.size(0)
        training_loss /= len(train_iterator)
 
        
        model.eval()
        for batch_idx,batch in enumerate(valid_iterator):
            predict = model(batch.tweet)
            loss = criterion(predict,batch.label)
            valid_loss += loss.data.item() * batch.tweet.size(0)
 
        valid_loss /= len(valid_iterator)
        print('Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}'.format(epoch, training_loss, valid_loss))

In [20]:
train(15, model, optimizer, criterion, train_iterator, valid_iterator)        

Epoch: 1, Training Loss: 23.12, Validation Loss: 13.70
Epoch: 2, Training Loss: 22.92, Validation Loss: 13.81
Epoch: 3, Training Loss: 22.71, Validation Loss: 12.49
Epoch: 4, Training Loss: 22.24, Validation Loss: 12.35
Epoch: 5, Training Loss: 22.24, Validation Loss: 13.15
Epoch: 6, Training Loss: 22.33, Validation Loss: 13.23
Epoch: 7, Training Loss: 22.08, Validation Loss: 13.20
Epoch: 8, Training Loss: 22.19, Validation Loss: 13.53
Epoch: 9, Training Loss: 22.23, Validation Loss: 13.16
Epoch: 10, Training Loss: 22.23, Validation Loss: 13.10
Epoch: 11, Training Loss: 22.23, Validation Loss: 13.18
Epoch: 12, Training Loss: 22.15, Validation Loss: 12.45
Epoch: 13, Training Loss: 22.02, Validation Loss: 13.61
Epoch: 14, Training Loss: 22.02, Validation Loss: 12.30
Epoch: 15, Training Loss: 22.03, Validation Loss: 12.60


# Сохранение и загрузка модели

Мы можем либо сохранить всю модель, используя `save`, либо только параметры, используя `state_dict`. 
Использование последнего обычно предпочтительнее, поскольку оно позволяет повторно использовать параметры, даже если структура модели изменяется (или применять параметры от одной модели к другой).

In [21]:
# path for model save
models_dir = "/gdrive/MyDrive/Colab_Notebooks/text_classification/models_rnn"


In [23]:
# Saving Model as Book Chapter 4

# cохранение текущих параметров и структуры модели
torch.save(model, models_dir+"/rnn-model")     
# это сохранение содержит карты параметров каждого слоя в модели
torch.save(model.state_dict(), models_dir+"/rnn-model-map-2")    


In [24]:
# создание модели
# rnn2 = model() 
# загрузка сохранённой модели
rnn2 = torch.load(models_dir+"/rnn-model")
# загрузка карты параметров слоёв модели в переменную rnn2_state_dict
rnn2_state_dict = torch.load(models_dir+"/rnn-model-map-2")
# загрузка карты параметров слоёв модели в модель rnn2
rnn2.load_state_dict(rnn2_state_dict)             

<All keys matched successfully>

## Making predictions

In [25]:
# model = torch.load(models_dir+"/rnn-model")

In [26]:
%%time

def classify_tweet(tweet):
    categories = {0: "Негатив", 1:"Позитив"}
    processed = TWEET.process([TWEET.preprocess(tweet)]) # preprocess() выполняет токенизацию на основе spaCy
                                                         # process создаёт из токенов тензор
    processed = processed.to(device)                     
    # model.eval()
    # return categories[model(processed).argmax().item()]
    rnn2.eval()
    return categories[rnn2(processed).argmax().item()]
'''
Тензорный элемент с наибольшим значением соответствует выбранному классу модели, 
поэтому мы используем argmax() для получения его индекса, 
а затем item() для преобразования этого тензора нулевой размерности в целое число Python, 
которое мы индексируем в нашем словаре categories.
'''

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.2 µs


In [27]:
tweet = "I love my country"
classify_tweet(tweet)

'Позитив'

In [28]:
tweet = "he hates dogs"
classify_tweet(tweet)

'Позитив'

In [29]:
tweet = "sun is smiling"
classify_tweet(tweet)

'Негатив'

In [30]:
tweet = "I abuse you"
classify_tweet(tweet)

'Позитив'

In [31]:
tweet = "Russian President met with US President"
classify_tweet(tweet)

'Негатив'

In [32]:
tweet = "US announced sanctions on Russia"
classify_tweet(tweet)

'Негатив'

In [33]:
tweet = "an explosion thundered in the city"
classify_tweet(tweet)

'Позитив'

In [34]:
tweet = "Nice weather in Paris"
classify_tweet(tweet)

'Позитив'

In [35]:
tweet = "The war is over and there is peace"
classify_tweet(tweet)

'Позитив'

## Data Augmentation

In [36]:
def random_deletion(words, p=0.5):
    if len(words) == 1:
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words))
    if len(remaining) == 0:
        return [random.choice(words)]
    else:
        return remaining

In [37]:
def random_swap(sentence, n=5):
    length = range(len(sentence))
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1]
    return sentence

In [38]:
# Note: you'll have to define remove_stopwords() and get_synonyms() elsewhere

def random_insertion(sentence,n):
    words = remove_stopwords(sentence)
    for _ in range(n):
        new_synonym = get_synonyms(random.choice(words))
        sentence.insert(randrange(len(sentence)+1), new_synonym)
    return sentence

In [11]:
!pwd
!mkdir /content/dataset
!cd /content/dataset/
!pwd
!ls

/content
mkdir: cannot create directory ‘/content/dataset’: File exists
/content
dataset  ESC-50  sample_data


In [8]:
!git clone https://github.com/karoldvl/ESC-50


Cloning into 'ESC-50'...
remote: Enumerating objects: 4154, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 4154 (delta 10), reused 0 (delta 0), pack-reused 4136
Receiving objects: 100% (4154/4154), 878.78 MiB | 29.07 MiB/s, done.
Resolving deltas: 100% (257/257), done.
Checking out files: 100% (2011/2011), done.


In [2]:
# Install googletrans version 3.1.0a0 (temporary fix for #57)
!pip install googletrans==3.1.0a0

     |████████████████████████████████| 61kB 9.8MB/s 
     |████████████████████████████████| 51kB 8.8MB/s 
     |████████████████████████████████| 1.2MB 36.2MB/s 
     |████████████████████████████████| 71kB 12.1MB/s 
     |████████████████████████████████| 61kB 5.1MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-cp37-none-any.whl size=16368 sha256=0fdbe03d7912224dc60405cc340034f068e253c1de4215a79c8f5c09d48ca8b1
  Stored in directory: /root/.cache/pip/wheels/27/7a/a0/aff3babbb775549ce6813cb8fa7ff3c0848c4dc62c20f8fdac
Successfully built googletrans


In [4]:
import googletrans
import random

translator = googletrans.Translator()

sentences = ['The cat sat on the mat']

translations_fr = translator.translate(sentences, dest='fr')
fr_text = [t.text for t in translations_fr] 
print(fr_text)   
translations_en = translator.translate(fr_text, dest='en')
en_text = [t.text for t in translations_en]
print(en_text)   

available_langs = list(googletrans.LANGUAGES.keys())
tr_lang = random.choice(available_langs)
print(f"Translating to {googletrans.LANGUAGES[tr_lang]}")

translations = translator.translate(sentences, dest=tr_lang)
t_text = [t.text for t in translations]
print(t_text)

translations_en_random = translator.translate(t_text, src=tr_lang, dest='en')
en_text = [t.text for t in translations_en_random]
print(en_text)

["Le chat s'est assis sur le tapis"]
['The cat sat on the carpet']
Translating to xhosa
['Ikati yayihleli emethini']
['The cat was sitting on the mat']
